In [ ]:
!pip install keras-tuner
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [4]:
df=pd.read_csv('/content/Real_Combine.csv')

In [23]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

hp-->‘hp’ is an object internally passed by tuner to model-building function to help it specify range of hyperparameter values whenever the tuner is called.

hp.Int->number of layers ranging between 32 and 512 within a gap of 32.

hp.Choice-> Specifies values or functions to be used for hyperparameter tuning

In [19]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Air Quality Index1')

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [24]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

With random_state=0 , we get the same train and test sets across different executions. With random_state=42, we get the same train and test sets across different executions, but in this time, the train and test sets are different from the previous case with random_state=0 . random_state=None different_results

In [22]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [25]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 09s]
val_mean_absolute_error: 49.83958435058594

Best val_mean_absolute_error So Far: 48.59799830118815
Total elapsed time: 00h 00m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [26]:
tuner.results_summary()

Results summary
Results in project1/Air Quality Index1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 4
units_0: 160
units_1: 224
learning_rate: 0.01
units_2: 64
units_3: 416
units_4: 160
units_5: 480
units_6: 64
units_7: 224
units_8: 192
units_9: 128
units_10: 416
units_11: 32
units_12: 64
units_13: 160
Score: 48.59799830118815
Trial summary
Hyperparameters:
num_layers: 9
units_0: 160
units_1: 480
learning_rate: 0.01
units_2: 288
units_3: 96
units_4: 128
units_5: 192
units_6: 256
units_7: 64
units_8: 288
units_9: 384
units_10: 320
units_11: 64
units_12: 480
units_13: 64
Score: 49.83958435058594
Trial summary
Hyperparameters:
num_layers: 14
units_0: 64
units_1: 192
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
Score: 54.07647832234701
Trial summary
Hyperparameters:
num_layers: 10
units_0: 32
units_1: 256
learning_rate: 0.01
units_2: 384
units_3: 1

The Keras Tuner sometimes throws error “tensorflow:Oracle triggered exit” when we try to rerun the code. Changing this directory name helps us to avoid the error in rerun.

References https://medium.com/swlh/hyperparameter-tuning-in-keras-tensorflow-2-with-keras-tuner-randomsearch-hyperband-3e212647778f